In [3]:
!pip install fastai --upgrade --use-feature=2020-resolver


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --use-feature


In [4]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 31.6MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 1.2MB 50.2MB/s 
     |████████████████████████████████| 204kB 50.4MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
Mounted at /content/gdrive


In [5]:
from fastai.vision.all import *
from fastbook import *

In [6]:
path = untar_data(URLs.MNIST)

In [7]:
Path.BASE_PATH = path

In [8]:
path.ls()

(#2) [Path('testing'),Path('training')]

In [9]:
(path/'training').ls()

(#10) [Path('training/0'),Path('training/9'),Path('training/8'),Path('training/4'),Path('training/6'),Path('training/5'),Path('training/2'),Path('training/7'),Path('training/3'),Path('training/1')]

In [10]:
nines = (path/'training'/'9').ls().sorted()

In [11]:
img_path = nines[1]
img = Image.open(img_path)
img

In [12]:
img_t = tensor(img)
df = pd.DataFrame(img_t)
df.style.set_properties(**{'font-size':'7pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,33,54,151,151,194,151,151,73,48,14,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,64,237,254,254,254,254,254,254,254,254,182,48,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,12,187,255,173,114,69,10,10,10,89,169,244,241,61,0,0,0,0,0,0,0,0


In [13]:
train_image_list = get_image_files(path/'training')
train_x_tensor = [tensor(Image.open(img_path)) for img_path in train_image_list]
train_y_list = [int(img_path.parent.name) for img_path in train_image_list]

In [14]:
train_x = (torch.stack(train_x_tensor).float()/255).view(-1,28*28)
train_y = (tensor(train_y_list)).view(-1,1)

In [15]:
train_x.shape,train_y.shape

(torch.Size([60000, 784]), torch.Size([60000, 1]))

In [16]:
valid_image_list = get_image_files(path/'testing')
valid_x_tensor = [tensor(Image.open(img_path)) for img_path in valid_image_list]
valid_y_list = [int(img_path.parent.name) for img_path in valid_image_list]

In [17]:
valid_x = (torch.stack(valid_x_tensor).float()/255).view(-1,28*28)
valid_y = (tensor(valid_y_list)).view(-1,1)

In [18]:
valid_x.shape,valid_y.shape

(torch.Size([10000, 784]), torch.Size([10000, 1]))

In [19]:
train_dset = list(zip(train_x,train_y))
valid_dset = list(zip(train_x,train_y))

In [47]:
simple_net = nn.Sequential(
    nn.Linear(28*28,100),
    nn.ReLU(),
    nn.Linear(100,30),
    nn.ReLU(),
    nn.Linear(30,10)
    )

In [66]:
def mnist_loss(predictions, targets):
    logsoftmax = nn.LogSoftmax(dim=1)
    loss = nn.NLLLoss()
    return loss(logsoftmax(predictions),targets.squeeze())
    # loss = nn.CrossEntropyLoss()
    # return loss(predictions, targets.squeeze())

In [22]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()
    return loss

In [50]:
def batch_accuracy(predictions, targets):
    predictions = F.softmax(predictions)
    digit = predictions.max(dim=1)[1]
    return (digit == targets).float().mean()

In [24]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

In [25]:
class BasicOptim:
    def __init__(self,params,lr):
        self.params,self.lr = list(params),lr
        
    def step(self, *args, **kwargs):
        for p in self.params:
            p.data -= p.grad.data * self.lr
            
    def zero_grad(self, *args, **kwargs):
        for p in self.params:
            p.grad = None

In [26]:
def train_epoch(model):
    for xb,yb in train_dl:
        calc_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()

In [27]:
def train_model(model,epochs):
    for i in range(epochs):
        train_epoch(model)
        print('epoch',i, ':',validate_epoch(model))

In [58]:
train_dl = DataLoader(train_dset, batch_size=64)
# valid_dl = DataLoader(valid_dset, batch_size=256)

In [67]:
opt = BasicOptim(simple_net.parameters(), lr= 0.04)

In [68]:
train_model(simple_net,60)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


epoch 0 : 0.9534
epoch 1 : 0.9536
epoch 2 : 0.9538
epoch 3 : 0.9538
epoch 4 : 0.9541
epoch 5 : 0.9543
epoch 6 : 0.9546
epoch 7 : 0.9548
epoch 8 : 0.9551
epoch 9 : 0.9552
epoch 10 : 0.9556
epoch 11 : 0.9558
epoch 12 : 0.9562
epoch 13 : 0.9562
epoch 14 : 0.9565
epoch 15 : 0.9567
epoch 16 : 0.9569
epoch 17 : 0.957
epoch 18 : 0.9574
epoch 19 : 0.9575
epoch 20 : 0.9578
epoch 21 : 0.958
epoch 22 : 0.9582
epoch 23 : 0.9584
epoch 24 : 0.9586
epoch 25 : 0.9588
epoch 26 : 0.959
epoch 27 : 0.9593
epoch 28 : 0.9595
epoch 29 : 0.9597
epoch 30 : 0.9598
epoch 31 : 0.96
epoch 32 : 0.9603
epoch 33 : 0.9604
epoch 34 : 0.9606
epoch 35 : 0.9609
epoch 36 : 0.9611
epoch 37 : 0.9613
epoch 38 : 0.9615
epoch 39 : 0.9616
epoch 40 : 0.9619
epoch 41 : 0.9621
epoch 42 : 0.9623
epoch 43 : 0.9624
epoch 44 : 0.9626
epoch 45 : 0.9627
epoch 46 : 0.9629
epoch 47 : 0.963
epoch 48 : 0.9633
epoch 49 : 0.9635
epoch 50 : 0.9634
epoch 51 : 0.9636
epoch 52 : 0.9639
epoch 53 : 0.964
epoch 54 : 0.964
epoch 55 : 0.9642
epoch 56 :